In [1]:
"""
Baseline for CIL project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image
from random import shuffle

import code

import tensorflow.python.platform

import numpy as np
import tensorflow as tf

import cil_helper as cil

from skimage.feature import greycomatrix, greycoprops

In [2]:
try:
    tf.app.flags.DEFINE_string('log_dir', '/tmp/tensorflow_lukas_balz',
                           """Directory where to write event logs """
                           """and checkpoint.""")
except: 
    print(tf.app.flags.FLAGS.log_dir)
    
FLAGS = tf.app.flags.FLAGS

In [3]:
TRAINING_SIZE = 90
VALIDATION_SIZE = 10  # Size of the validation set.
TEST_SIZE = 50
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 256 # 64
NUM_EPOCHS = 40
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 120

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 2

# all prediction size
ALL_PER = 0.3

In [4]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 
test_dir = "test/"

In [5]:
# returns all train images in rgb
def get_train_imgs():
    imgs = []
    for i in range(TRAINING_SIZE):
        imageid = "satImage_%.3d" % (i+1)
        image_filename = train_data_filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            raise Exception('File ' + image_filename + ' does not exist')
    return np.asarray(imgs)

In [6]:
# returns all test images as rgb
def get_test_imgs():
    imgs = []
    for i in range(TEST_SIZE):
        imagename = "test_" + str(i+1)
        image_filename = test_dir + imagename + "/" + imagename + ".png"
        if os.path.isfile(image_filename):
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            raise Exception('File ' + image_filename + ' does not exist')
    return np.asarray(imgs)

In [7]:
# returns a numpy array [Saturation, Lightness] of a pixel
def sat_light(rgb):
    Cmax = rgb.max()
    Cmin = rgb.min()
    d = Cmax - Cmin
    L = (Cmax + Cmin) / 2
    S = d / (1 - (abs(2 * L - 1)))
    if np.isnan(S):
        S = 0
    return np.asarray([S, L], dtype=np.float32)

In [8]:
max_offset = 1 #feel free to change
LIGHTNESS_LEVELS = 8
LIGHTNESS_IDX = 4
#returns [contrast, correlation, energy, homogeneity]
def texture_features(image, y, x):
    ylow = max(0, y-max_offset)
    yhi = min(image.shape[0], y+max_offset+1)
    xlow = max(0, x-max_offset)
    xhi = min(image.shape[1], x+max_offset+1)
    #print([ylow,yhi,xlow,xhi])
    window = ((image[ylow:yhi, xlow:xhi, LIGHTNESS_IDX]) * (LIGHTNESS_LEVELS-1)).astype(np.uint8)
    try:
        glcm = greycomatrix(window, [i for i in range(1,max_offset+1)], [0, np.pi/2], levels=LIGHTNESS_LEVELS, normed=True)
    except AssertionError:
        print(repr(window))
        print(repr(image))
        raise
    contrast = greycoprops(glcm, 'contrast').mean()
    correlation = greycoprops(glcm, 'correlation').mean()
    energy = greycoprops(glcm, 'energy').mean()
    homogeneity = greycoprops(glcm, 'homogeneity').mean()
    #dissimilarity = greycoprops(glcm, 'dissimilarity').mean()
    return np.asarray([contrast, correlation, energy, homogeneity], dtype=np.float32)

In [9]:
NUM_CHANNELS = 9 # RGB, Saturation, Lightness, 4 Texture
def preprocess(img):
    new_shape = list(img.shape)
    new_shape[2] = NUM_CHANNELS
    result = np.empty(new_shape, dtype=np.float32)
    result[:,:,:3] = img
    for y in range(len(img)):
        for x in range(len(img[y])):
            result[y,x,3:5] = sat_light(result[y,x,:3])
    #print('Saturation and lightness computed.')
    for y in range(len(img)):
        for x in range(len(img[y])):
            result[y,x,5:] = texture_features(result[:,:,:5], y, x)
    #print('Texture deatures computed.')
    return result

In [10]:
# Extract label images
def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [cil.img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)

In [11]:
def make_patches(imgs):
    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    DIMS = 4
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [cil.img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return np.asarray(data)

In [12]:
# takes an array of images, preprocesses them and stores the result
# in save_dir/preprocessed.npy
def preprocess_imgs(imgs, save_dir):
    assert save_dir.endswith('/')
    processed = []
    for i in range(len(imgs)):
        processed.append(preprocess(imgs[i]))
        print('Image ' + str(i) + ' preprocessed.')
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    processed_np = np.asarray(processed)
    np.save(save_dir + 'preprocessed.npy', processed_np)
    return processed_np

In [13]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.75 # percentage of pixels > 1 required to assign a foreground label to a patch
    #ret#urn [1-v , v]
    df = np.sum(v)
    if df < foreground_threshold:
       return [0, 1]
    else:
       return [1, 0]

In [14]:
PREPROCESS_TRAIN_DATA = False
PREPROCESS_TEST_DATA = False

if PREPROCESS_TRAIN_DATA:
    preprocess_imgs(get_train_imgs(), train_data_filename)
if PREPROCESS_TEST_DATA:
    preprocess_imgs(get_test_imgs(), test_dir)

In [15]:
def load_preproc(save_dir):
    assert save_dir.endswith('/')
    return np.load(save_dir + 'preprocessed.npy')

In [16]:
train_preproc = load_preproc(train_data_filename)
train_data = make_patches(train_preproc)

train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

In [17]:
#print(train_data.shape)
#NUM_CHANNELS = 5
#train_data = train_data[:,:,:,0:NUM_CHANNELS]
print(train_data.shape)

(4000000, 2, 2, 9)


In [18]:
c0 = 0
c1 = 0
for i in range(len(train_labels)):
    if train_labels[i][0] > 0.5:
        c0 = c0 + 1 # foreground -> road
    else:
        c1 = c1 + 1 # background
print ('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 678904 c1 = 2921096


In [19]:
sampling = False

In [20]:
print ('Balancing training data...')
min_c = min(c0, c1)
max_c = max(c0, c1)
idx0 = [i for i, j in enumerate(train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(train_labels) if j[1] == 1]

if sampling: #sample smaller class
    if c0 < c1:
        while len(idx0) < c1:
            print(len(idx0))
            idx0.extend(idx0)
            print(len(idx0))
    else:
        while len(idx1)<c0:
            idx1.extend(idx1)
    lim = max_c
else: 
    lim = min_c
    
new_indices = idx0[0:lim] + idx1[0:lim]
train_data = train_data[new_indices,:,:,:]
train_labels = train_labels[new_indices]
    
print(len(new_indices))
print(train_data.shape)

Balancing training data...
1357808
(1357808, 2, 2, 9)


In [21]:
train_size = train_labels.shape[0]

In [22]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS),
    name='train_data')
train_labels_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, cil.NUM_LABELS),
    name='train_labels')
train_all_data_node = tf.constant(train_data)

In [23]:
# The variables below hold all the trainable weights. They are passed an
# initial value which will be assigned when when we call:
# {tf.initialize_all_variables().run()}
conv1_weights = cil.weight_variable([2, 2, NUM_CHANNELS, 56])
conv1_biases = cil.bias_variable([56])

fc1_weights = tf.Variable(  # fully connected, depth 512.
    tf.truncated_normal([56, 400],
                        stddev=0.1,
                        seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[400]))
fc2_weights = tf.Variable(
    tf.truncated_normal([400, cil.NUM_LABELS],
                        stddev=0.1,
                        seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[cil.NUM_LABELS]))
conv1_weights_hist = tf.histogram_summary("conv1_weights", conv1_weights)
conv1_biases_hist = tf.histogram_summary("conv1_biases", conv1_biases)
fc1_weights_hist = tf.histogram_summary("fc1_weights", fc1_weights)
fc1_biases_hist = tf.histogram_summary("fc1_biases", fc1_biases)
fc2_weights_hist = tf.histogram_summary("fc2_weights", fc2_weights)
fc2_biases_hist = tf.histogram_summary("fc2_biases", fc2_biases)

In [24]:
# We will replicate the model structure for the training subgraph, as well
# as the evaluation subgraphs, while sharing the trainable parameters.
def model(data, train=False):
    """The Model definition."""
    conv = cil.conv2d(data, conv1_weights)
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    pool = cil.max_pool_2x2(relu)

    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(
        pool,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    
    print(pool.get_shape())
    print(reshape.get_shape())
    print(fc1_weights.get_shape())
    
    fc1 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    if train:
        fc1 = tf.nn.dropout(fc1, 0.5, seed=SEED)
    fc2 = tf.matmul(fc1, fc2_weights) + fc2_biases

    if train == True:
        summary_id = '_0'
        s_data = cil.get_image_summary(data)
        filter_summary0 = tf.image_summary('summary_data' + summary_id, s_data)
        s_conv = cil.get_image_summary(conv)
        filter_summary2 = tf.image_summary('summary_conv' + summary_id, s_conv)
        #s_pool = cil.get_image_summary(pool)
        #filter_summary3 = tf.image_summary('summary_pool' + summary_id, s_pool)

    return fc2

In [25]:
# Training computation: logits + cross-entropy loss.
logits = model(train_data_node, True) # BATCH_SIZE*NUM_LABELS
# print 'logits = ' + str(logits.get_shape()) + ' train_labels_node = ' + str(train_labels_node.get_shape())
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits, train_labels_node))
#loss = tf.reduce_mean(tf.square(logits - train_labels_node))
loss_summary = tf.scalar_summary('loss', loss)

(256, 1, 1, 56)
(256, 56)
(56, 400)


In [26]:
all_params_node = [conv1_weights, conv1_biases, fc1_weights, fc1_biases, fc2_weights, fc2_biases]
all_params_names = ['conv1_weights', 'conv1_biases', 'conv2_weights', 'conv2_biases', 'fc1_weights', 'fc1_biases', 'fc2_weights', 'fc2_biases']
all_grads_node = tf.gradients(loss, all_params_node)
all_grad_norms_node = []
for i in range(0, len(all_grads_node)):
    norm_grad_i = tf.global_norm([all_grads_node[i]])
    all_grad_norms_node.append(norm_grad_i)
    all_params_summary = tf.scalar_summary(all_params_names[i], norm_grad_i)

In [27]:
# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

In [28]:
# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    0.01,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)
learning_rate_summary = tf.scalar_summary('learning_rate', learning_rate)

In [29]:
# Use simple momentum for the optimization.
#optimizer = tf.train.MomentumOptimizer(learning_rate, 0.0).minimize(loss, global_step=batch)

# try adam
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=batch)

In [30]:
# Predictions for the minibatch, validation set and test set.
train_prediction = tf.nn.softmax(logits)
# We'll compute them only once in a while by calling their {eval()} method.
all_shape = train_all_data_node.get_shape()
all_ind = int(int(all_shape[0])*ALL_PER)
train_all_prediction = tf.nn.softmax(model(train_all_data_node[0:all_ind,:,:,:]))

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

(407342, 1, 1, 56)
(407342, 56)
(56, 400)


In [31]:
s = tf.InteractiveSession()

In [32]:
# Get prediction overlaid on the original image for given input image
def get_prediction_with_overlay(img):
    
    img_prediction = get_prediction(img)
    oimg = cil.make_img_overlay(img[:,:,:3], img_prediction)

    return oimg

# Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(img):
    
    img_prediction = get_prediction(img)
    cimg = cil.concatenate_images(img[:,:,:3], img_prediction)

    return cimg

# Get prediction for given input image 
def get_prediction(img):
    data = np.asarray(cil.img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    data = data[:,:,:,0:NUM_CHANNELS]
    data_node = tf.constant(data)
    output = tf.nn.softmax(model(data_node))
    output_prediction = s.run(output)
    img_prediction = cil.label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

In [33]:
if RESTORE_MODEL:
    # Restore variables from disk.
    saver.restore(s, FLAGS.train_dir + "/model2.ckpt")
    print("Model restored.")

In [34]:
# Run all the initializers to prepare the trainable parameters.
tf.initialize_all_variables().run()

# Build the summary operation based on the TF collection of Summaries.
summary_op = tf.merge_summary([loss_summary, learning_rate_summary, all_params_summary, conv1_weights_hist])
summary_writer = tf.train.SummaryWriter(FLAGS.log_dir,
                                                graph=s.graph)
print ('Initialized!')

Initialized!


In [35]:
# Loop through training steps.
print ('Total number of iterations = ' + str(int(NUM_EPOCHS * train_size / BATCH_SIZE)))

training_indices = range(train_size)

for iepoch in range(NUM_EPOCHS):

    # Permute training indices
    perm_indices = np.random.permutation(training_indices)

    for step in range (int(train_size / BATCH_SIZE)):

        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_indices = perm_indices[offset:(offset + BATCH_SIZE)]

        # Compute the offset of the current minibatch in the data.
        # Note that we could use better randomization across epochs.
        batch_data = train_data[batch_indices, :, :, :]
        batch_labels = train_labels[batch_indices]
        
        #print('batch_data.shape = ' + str(batch_data.shape))
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}

        if step % RECORDING_STEP == 0:

            summary_str, _, l, lr, predictions = s.run(
                        [summary_op, optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
            
            l, lr, all_predictions = s.run(
                        [loss, learning_rate, train_all_prediction],
                        feed_dict=feed_dict)
            # print_predictions(predictions, batch_labels)

            #print('Epoch: ' + str(iepoch))
            #print('Step: ' + str(step))
            #print ('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            #print ('Minibatch error: %.1f%%' % cil.error_rate(predictions,
                                                              #   batch_labels))
            print('Step: ' + str(step + iepoch*int(train_size / BATCH_SIZE)) + ' error: %.1f%%' % cil.error_rate(all_predictions, train_labels[0:all_ind]))
            #print(batch_labels)
            sys.stdout.flush()
        else:
            # Run the graph and fetch some of the nodes.
            _, l, lr, predictions = s.run(
                        [optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)

# Save the variables to disk.
#save_path = saver.save(s, FLAGS.train_dir + "/model.ckpt")
#print("Model saved in file: %s" % save_path) 

Total number of iterations = 212157
Step: 0 error: 20.0%
Step: 120 error: 28.6%
Step: 240 error: 37.9%
Step: 360 error: 29.1%
Step: 480 error: 40.2%
Step: 600 error: 40.2%
Step: 720 error: 38.3%
Step: 840 error: 36.0%
Step: 960 error: 36.8%
Step: 1080 error: 38.9%
Step: 1200 error: 40.2%
Step: 1320 error: 32.1%
Step: 1440 error: 29.1%
Step: 1560 error: 41.3%
Step: 1680 error: 32.1%
Step: 1800 error: 33.4%
Step: 1920 error: 39.0%
Step: 2040 error: 29.4%
Step: 2160 error: 38.9%
Step: 2280 error: 38.1%
Step: 2400 error: 28.7%
Step: 2520 error: 37.8%
Step: 2640 error: 23.8%
Step: 2760 error: 32.2%
Step: 2880 error: 20.3%
Step: 3000 error: 23.9%
Step: 3120 error: 28.2%
Step: 3240 error: 29.1%
Step: 3360 error: 25.8%
Step: 3480 error: 30.1%
Step: 3600 error: 33.6%
Step: 3720 error: 28.1%
Step: 3840 error: 27.0%
Step: 3960 error: 20.1%
Step: 4080 error: 38.7%
Step: 4200 error: 32.6%
Step: 4320 error: 42.7%
Step: 4440 error: 33.2%
Step: 4560 error: 35.3%
Step: 4680 error: 26.0%
Step: 4800 erro

In [36]:
save_path = saver.save(s, FLAGS.train_dir + "/model2.ckpt")

In [37]:
print ("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
    
for i in range(TRAINING_SIZE):
    pimg = get_prediction_with_groundtruth(train_preproc[i])
    Image.fromarray(pimg).save(prediction_training_dir + "prediction2_" + str(i+1) + ".png")
    oimg = get_prediction_with_overlay(train_preproc[i])
    oimg.save(prediction_training_dir + "overlay2_" + str(i+1) + ".png")  

Running prediction on training set
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(40000, 1, 1, 56)
(40000, 56)
(56, 400)
(4000

In [38]:
print ("Running prediction on test set")
prediction_test_dir = "predictions_test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)

test_preproc = load_preproc(test_dir)
for i in range(TEST_SIZE):
    pred = get_prediction(test_preproc[i])
    pimg = cil.img_float_to_uint8(pred)
    Image.fromarray(pimg).save(prediction_test_dir + "prediction2_" + str(i+1) + ".png")
    
    oimg = cil.make_img_overlay(test_preproc[i], pred)
    oimg.save(prediction_test_dir + "overlay2_" + str(i+1) + ".png")

Running prediction on test set
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1, 1, 56)
(92416, 56)
(56, 400)
(92416, 1

In [39]:
import re
th = 0.49

In [40]:
# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > th:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\_\d+", image_filename).group(0)[1:])
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


def save(submission_filename):
    #subm#ission_filename = 'submission_07.csv'
    image_filenames = []
    for i in range(1, 51):
        imagename = 'prediction2_' + str(i)
        image_filename = 'predictions_test/' + imagename + '.png'
        print(image_filename)
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)
    
save('submission_lukas_2x2.csv')

predictions_test/prediction2_1.png
predictions_test/prediction2_2.png
predictions_test/prediction2_3.png
predictions_test/prediction2_4.png
predictions_test/prediction2_5.png
predictions_test/prediction2_6.png
predictions_test/prediction2_7.png
predictions_test/prediction2_8.png
predictions_test/prediction2_9.png
predictions_test/prediction2_10.png
predictions_test/prediction2_11.png
predictions_test/prediction2_12.png
predictions_test/prediction2_13.png
predictions_test/prediction2_14.png
predictions_test/prediction2_15.png
predictions_test/prediction2_16.png
predictions_test/prediction2_17.png
predictions_test/prediction2_18.png
predictions_test/prediction2_19.png
predictions_test/prediction2_20.png
predictions_test/prediction2_21.png
predictions_test/prediction2_22.png
predictions_test/prediction2_23.png
predictions_test/prediction2_24.png
predictions_test/prediction2_25.png
predictions_test/prediction2_26.png
predictions_test/prediction2_27.png
predictions_test/prediction2_28.png
p